###Run pre-trained BERT model for text classification

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import trange
import numpy as np
import matplotlib.pyplot as plt
import os
from jx_dev.utils_text_clf import utils_text_clf as utils
import pandas as pd

device = torch.device("cpu")
n_gpu = torch.cuda.device_count()

Load data

In [ ]:
data_dir = os.path.dirname(os.getcwd()) + '/data'
filename = 'train.jsonl'
file = os.path.join(data_dir, filename)  # .json file
train_df = utils.parse_json(file)

test_filename = 'test.jsonl'
test_file = os.path.join(data_dir, test_filename)  # .json file
test_df = utils.parse_json(test_file)

Format data for feeding into BERT model

In [ ]:
train_tweets = train_df.response.values
# We need to add special tokens at the beginning and end of each tweet for BERT to work properly
train_tweets = ["[CLS] " + tweet + " [SEP]" for tweet in train_tweets]
train_df["label"] = (train_df.label == 'SARCASM').astype(int)
train_labels = train_df.label.values

Tokenize training data using BERT pre-trained tokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_tweets = [tokenizer.tokenize(tweet) for tweet in train_tweets]

Recommended make length of squences of tokens, each sequence is either padded or cut to fit this length

In [ ]:
MAX_LEN = 128

In [ ]:
# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(tweet) for tweet in tokenized_tweets]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

Attention Masks to indicate what is part of the tweet and what is padding

In [ ]:
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
    seq_mask = [float(i > 0) for i in seq]
    attention_masks.append(seq_mask)

Split train data into train and validation for fitting the model

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(input_ids, train_labels, random_state=42, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=42, test_size=0.1)

Convert all data into torch tensors, the required datatype for the model

In [ ]:
x_train = torch.tensor(x_train)
x_val = torch.tensor(x_val)
y_train = torch.tensor(y_train)
y_val = torch.tensor(y_val)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

Create batches for training and validation

In [ ]:
batch_size = 32
x_train = TensorDataset(x_train, train_masks, y_train)
train_sampler = RandomSampler(x_train)
train_dataloader = DataLoader(x_train, sampler=train_sampler, batch_size=batch_size)

x_val = TensorDataset(x_val, validation_masks, y_val)
validation_sampler = SequentialSampler(x_val)
validation_dataloader = DataLoader(x_val, sampler=validation_sampler, batch_size=batch_size)

Load model

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Create optimizer

In [ ]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]
optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=2e-5,
                     warmup=.1)

Accuracy function for optimization (not used)

In [ ]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

Optimize using F1

In [ ]:
def flat_f1(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    tp = float(sum([pred_flat[i] for i in range(len(pred_flat)) if (pred_flat[i] == 1) & (labels_flat[i] == 1)]))
    fp = float(sum([pred_flat[i] for i in range(len(pred_flat)) if (pred_flat[i] == 1) & (labels_flat[i] == 0)]))
    fn = float(sum([labels_flat[i] for i in range(len(labels_flat)) if (pred_flat[i] == 0) & (labels_flat[i] == 1)]))
    precision = tp/(tp + fp)
    recall = tp/(tp + fn)
    f1 = (2*precision*recall)/(precision + recall)
    return f1

Store predictions and loss/accuracy for plotting

In [ ]:
t = []
train_loss_set = []

Number of training epochs (authors recommend between 2 and 4)

In [ ]:
epochs = 4

Train model, capturing metrics at each epoch

In [ ]:
# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
    # Set our model to training mode (as opposed to evaluation mode)
    model.train()
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    
    # Train the data for one epoch
    for step, batch in enumerate(train_dataloader):
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Clear out the gradients (by default they accumulate)
        optimizer.zero_grad()
        # Forward pass
        loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        train_loss_set.append(loss.item())
        # Backward pass
        loss.backward()
        # Update parameters and take a step using the computed gradient
        optimizer.step()

        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    print("Train loss: {}".format(tr_loss / nb_tr_steps))

    # Validation

    # Put model in evaluation mode to evaluate loss on the validation set
    model.eval()

    # Tracking variables
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Telling the model not to compute or store gradients, saving memory and speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions
            logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_f1(logits, label_ids)

        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("Validation Accuracy: {}".format(eval_accuracy / nb_eval_steps))

Plot loss after train and validation

In [ ]:
plt.figure(figsize=(15, 8))
plt.title("Training loss")
plt.xlabel("Batch")
plt.ylabel("Loss")
plt.plot(train_loss_set)
plt.show()

Format test set for predictions

In [ ]:
test_tweets = test_df.response.values
# We need to add special tokens at the beginning and end of each tweet for BERT to work properly
test_tweets = ["[CLS] " + tweet + " [SEP]" for tweet in test_tweets]
# test_df["label"] = test_df.id.str[8::].astype(int)
test_labels = test_df.id.values

Tokenize test tweets in the same way as training data

In [ ]:
test_tokenized_tweets = [tokenizer.tokenize(tweet) for tweet in test_tweets]
MAX_LEN = 128
# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(tweet) for tweet in test_tokenized_tweets]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
    seq_mask = [float(i > 0) for i in seq]
    attention_masks.append(seq_mask)
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)

batch_size = 32
prediction_data = TensorDataset(prediction_inputs, prediction_masks)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

Evaluate model on test data

In [ ]:
model.eval()
# Tracking variables
predictions, true_labels = [], []

for batch in prediction_dataloader:
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up prediction
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    # Move logits to CPU
    logits = logits.detach().cpu().numpy()
    # Store predictions and true labels
    predictions.append(logits)

In [ ]:
print(predictions)

Flatten output to get predictions in right format

In [ ]:
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
print(flat_predictions)
print(test_labels)
output_dict = {"id": test_labels, "prediction": flat_predictions}
out_df = pd.DataFrame(output_dict)

Cast 0/1 to SARCASM and NOT_SARCASM for final output

In [ ]:
out_df["prediction"] = out_df.prediction.apply(lambda x: 'SARCASM' if x == 1 else 'NOT_SARCASM')

Save final predictions

In [ ]:
out_df.to_csv("answer.txt", index=False, header=False)
